<a href="https://colab.research.google.com/github/jahnavi-2116/AI-ML-projects/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install nltk


In [3]:
# Loading the data set
import pandas as pd
import nltk
file_path = '/content/offerings - Reviews_Preprocessed.csv'
df = pd.read_csv(file_path)

In [4]:
df.shape

(250, 8)

In [5]:
df.columns

Index(['id', 'name', 'date', 'review title', 'review', 'travel type', 'rating',
       'preprocessed_reviews'],
      dtype='object')

In [6]:
df_reviews = df[['review']]

In [7]:
df.head()


,id,name,date,review title,review,travel type,rating,preprocessed_reviews
0,113317,Casablanca Hotel Times Square,Dec 19,"Wow, fabulous, amazing hotel, great location, ...",Amazing Mother and Daughter time at Casablanca...,Traveled with family,5,amazing mother daughter time casablanca hotel ...
1,113317,Casablanca Hotel Times Square,Dec 17,Greatest boutique hotel in NYC,After staying at different hotels in NYC over ...,Traveled as a couple,5,staying different hotel nyc many year found tr...
2,113317,Casablanca Hotel Times Square,Dec 12,Great Hotel and Great Location,Fantastic boutique hotel with the greatest loc...,Traveled as a couple,5,fantastic boutique hotel greatest location . s...
3,113317,Casablanca Hotel Times Square,Dec 12,A Jewel in the Jungle,Great hotel and staff. Convenient to all thing...,Traveled as a couple,5,great hotel staff . convenient thing nyc . fri...
4,113317,Casablanca Hotel Times Square,Dec 12,Girls Trip in NYC at Christmas,This is our 2nd girls trip to NYC & at the Cas...,Traveled with friends,5,2nd girl trip nyc & casablanca ! ! love hotel ...


In [8]:
df.tail()
df.tail(10)

,id,name,date,review title,review,travel type,rating,preprocessed_reviews
240,1200103,Hotel Sorella CITYCENTRE,Dec 18,GREAT,My stay was the absolute BEST. Any little ques...,Traveled with family,5,"stay absolute best . little question , staff h..."
241,1200103,Hotel Sorella CITYCENTRE,Dec 11,Sandra from Loujisiana,The buffett was horrible no meat no potatoes j...,Traveled with family,2,"buffett horrible meat potato big pot egg , mus..."
242,1200103,Hotel Sorella CITYCENTRE,Dec 10,Fantastic experience!,Stayed here to attend company party at a city ...,Traveled on business,5,stayed attend company party city center restau...
243,1200103,Hotel Sorella CITYCENTRE,Dec 7,Old and Worn but A+ for location,The best thing about this hotel is the locatio...,Traveled on business,4,best thing hotel location . heart city centre ...
244,1200103,Hotel Sorella CITYCENTRE,Dec 4,Entertaining Stay,I loved the location. It’s very family oriente...,Traveled on business,5,loved location . ’ family oriented lot shoppin...
245,1200103,Hotel Sorella CITYCENTRE,Dec 4,"Loved the location very closed to shops, resta...",I am very particular on hotels I usually reque...,Traveled on business,5,particular hotel usually request omni st regis...
246,1200103,Hotel Sorella CITYCENTRE,Dec 1,Overpriced and poorly managed,"Issues in the room, sink clogged, no soap, too...",Traveled on business,4,"issue room , sink clogged , soap , took multip..."
247,1200103,Hotel Sorella CITYCENTRE,Nov 2023,Excellent comfort and service with added trans...,Really good experience staying at this excelle...,Traveled on business,5,really good experience staying excellent hotel...
248,1200103,Hotel Sorella CITYCENTRE,Nov 2023,Great hotel shuttle service,Mr Collins had provided exceptional service as...,Traveled on business,5,mr collins provided exceptional service brough...
249,1200103,Hotel Sorella CITYCENTRE,Nov 2023,Collins is so helpful!!,Collins helped us unload so many bags and ensu...,Traveled on business,5,collins helped u unload many bag ensured got r...


In [9]:
# finding missing values
missing_values = df_reviews.isnull().sum()

# display only missing values with columns
missing_values = missing_values[missing_values > 0]
print(missing_values)

Series([], dtype: int64)


In [10]:
# review column
df_reviews = df[['review']].copy()
print(df_reviews.head())

                                              review
0  Amazing Mother and Daughter time at Casablanca...
1  After staying at different hotels in NYC over ...
2  Fantastic boutique hotel with the greatest loc...
3  Great hotel and staff. Convenient to all thing...
4  This is our 2nd girls trip to NYC & at the Cas...


In [12]:
df_reviews ['cleaned'] = df_reviews['review'].str.lower()
print(df_reviews.head())

                                              review                                            cleaned
0  Amazing Mother and Daughter time at Casablanca...  amazing mother and daughter time at casablanca...
1  After staying at different hotels in NYC over ...  after staying at different hotels in nyc over ...
2  Fantastic boutique hotel with the greatest loc...  fantastic boutique hotel with the greatest loc...
3  Great hotel and staff. Convenient to all thing...  great hotel and staff. convenient to all thing...
4  This is our 2nd girls trip to NYC & at the Cas...  this is our 2nd girls trip to nyc & at the cas...


In [13]:
# remove punctuation and numbers
import re
df_reviews['cleaned'] = df_reviews['cleaned'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
print(df_reviews.head())

                                              review                                            cleaned
0  Amazing Mother and Daughter time at Casablanca...  amazing mother and daughter time at casablanca...
1  After staying at different hotels in NYC over ...  after staying at different hotels in nyc over ...
2  Fantastic boutique hotel with the greatest loc...  fantastic boutique hotel with the greatest loc...
3  Great hotel and staff. Convenient to all thing...  great hotel and staff convenient to all things...
4  This is our 2nd girls trip to NYC & at the Cas...  this is our nd girls trip to nyc  at the casab...


In [14]:
def map_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating <= 2:
            return 'negative'
    else:
            return 'neutral'




In [15]:
df['sentiment'] = df['rating'].apply(map_sentiment)

In [16]:
# removing missing rows
df_reviews = df_reviews.dropna(subset=['cleaned'])
print(df_reviews.head())
print(df_reviews.tail())

                                              review                                            cleaned
0  Amazing Mother and Daughter time at Casablanca...  amazing mother and daughter time at casablanca...
1  After staying at different hotels in NYC over ...  after staying at different hotels in nyc over ...
2  Fantastic boutique hotel with the greatest loc...  fantastic boutique hotel with the greatest loc...
3  Great hotel and staff. Convenient to all thing...  great hotel and staff convenient to all things...
4  This is our 2nd girls trip to NYC & at the Cas...  this is our nd girls trip to nyc  at the casab...
                                                review                                            cleaned
245  I am very particular on hotels I usually reque...  i am very particular on hotels i usually reque...
246  Issues in the room, sink clogged, no soap, too...  issues in the room sink clogged no soap took m...
247  Really good experience staying at this excelle...  re

In [17]:
df.columns

Index(['id', 'name', 'date', 'review title', 'review', 'travel type', 'rating',
       'preprocessed_reviews', 'sentiment'],
      dtype='object')

In [18]:
import nltk
import re
nltk.download('punkt')
stopwords = {'i', 'the', 'and', 'an', 'to', 'on', 'off', 'so', 'at', 'in'}

# Inspect the 'cleaned' column before processing
print("Head of 'cleaned' column before tokenization:")
print(df_reviews['cleaned'].head())
print("\nTail of 'cleaned' column before tokenization:")
print(df_reviews['cleaned'].tail())


# Use regular expression for tokenization and remove stopwords
df_reviews['cleaned'] = df_reviews['cleaned'].apply(lambda x: ' '.join([word for word in re.findall(r'\b\w+\b', x) if word.lower() not in stopwords]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Head of 'cleaned' column before tokenization:
0    amazing mother and daughter time at casablanca...
1    after staying at different hotels in nyc over ...
2    fantastic boutique hotel with the greatest loc...
3    great hotel and staff convenient to all things...
4    this is our nd girls trip to nyc  at the casab...
Name: cleaned, dtype: object

Tail of 'cleaned' column before tokenization:
245    i am very particular on hotels i usually reque...
246    issues in the room sink clogged no soap took m...
247    really good experience staying at this excelle...
248    mr collins had provided exceptional service as...
249    collins helped us unload so many bags and ensu...
Name: cleaned, dtype: object


In [19]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    df_reviews['cleaned'], df['sentiment'], test_size=0.2, random_state=42
)

In [22]:
# Vectorize text
vectorizer=TfidfVectorizer(max_features=1000)
X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)

In [23]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000)

In [24]:
#predictions
y_pred = model.predict(X_test_vec)

from sklearn.metrics import classification_report, confusion_matrix
print("classification report")
print(classification_report(y_test, y_pred))

print("confusion matrix")
print(confusion_matrix(y_test, y_pred))


classification report
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         4
     neutral       0.00      0.00      0.00         3
    positive       0.86      1.00      0.92        43

    accuracy                           0.86        50
   macro avg       0.29      0.33      0.31        50
weighted avg       0.74      0.86      0.80        50

confusion matrix
[[ 0  0  4]
 [ 0  0  3]
 [ 0  0 43]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
# Gradio interface
import gradio as gr
import re

def predict_sentiment(review_text):
  cleaned = re.sub(r'[a-z\s]', '', review_text.lower())

  #vectorize
  vectorized = vectorizer.transform([cleaned])

  prediction= model.predict(vectorized)
  return f"Predicted Sentiment: {prediction[0]}"


In [32]:
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=4, placeholder="Hotel Review Here"),
    outputs="text",
    title="Hotel Review Sentiment Analysis",
    description="Please feel free to enter a review and find out if its negative, neutral or positive."
)

#Launch gradio
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc8a976ad663d67a0b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The above project is a sentiment analysis from scratch using a dataset of hotel reviews. So I started by cleaning the text data through basic NLP preprocessing steps like converting all text to lowercase, removing punctuation and numbers, tokenzing the words, and filtering out common stopwords. I then tried to map the review ratings to sentiment labels like positive, neutral, and negative - based on their numerical values. After preprocessing, I used the TFID vectorization to convert the text into numerical format suitable for model training.

Now for the modeling part, I chose Logistic Regression because its simple, efficient, and works well with text data. I trained the model using the TFIDF vectors and then evaluated its performance using a classification report and confusion matrix. Once the model was performing as expected, I built an interactive web interface using Gradio. This interface allows users to input a hotel review and get back a predicted sentiment in real time. Overall, this project helped me understand the full NLP workflow—from raw text preprocessing and feature extraction to model training, evaluation, and deployment.